# Codificação de arquivos e operações pontuais

## Convertendo imagem

Importando bibliotecas e definindo variávies.

In [1]:
import cv2
import os
import numpy as np

file_name = "VisaoTrab1.tiff"
file_name_no_extension = os.path.splitext(file_name)[0]

Abrindo o arquivo {{file_name}} "VisaoTrab1.tiff".

In [2]:
img = cv2.imread(file_name)

Criando arquivos JPEG - 10, 50 e 90%

In [3]:
created_files = []
for quality in [10, 50, 90]:
    created_files += [f'{file_name_no_extension}_{quality}.jpg']
	cv2.imwrite(created_files[-1], img, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

Salvar imagem como TIFF - LZW.

In [11]:
created_files += [f'{file_name_no_extension}_LZW.tiff']
cv2.imwrite(created_files[-1], img, [5])
# cv2.imwrite(created_files[-1], img, [int(cv2.IMWRITE_TIFF_COMPRESSION), 5])
# cv2.imwrite(created_files[-1], img)

True

## Correção gama

Convertento a imagem para HSV.

In [5]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

Valor gamma e constante com o valor máximo para o canal H.

In [6]:
gamma = 1.5
channel_max = 179.0

Separando o canal H na variável channel_h.

In [7]:
channel_h = np.zeros(hsv.shape[:2])
for x in range(hsv.shape[0]):
	for y in range(hsv.shape[1]):
		channel_h[x, y] = hsv[x, y][0] / channel_max

Aplicando operação gamma sobre todo o canal H.

In [8]:
channel_h = channel_h * gamma * channel_max

Salvando na matrix.

In [9]:
for x in range(hsv.shape[0]):
	for y in range(hsv.shape[1]):
		hsv[x, y][0] = int(max(min(channel_h[x, y], channel_max), 0))

Convertendo imagem para BGR e salvando o arquivo.

In [10]:
img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
cv2.imwrite(f'{file_name_no_extension}_{gamma}.tiff', img)


True